In [1]:
import torch
from torch import nn
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
import pandas as pd

In [2]:
class Network(nn.Module):
    def __init__(self, p1, p2):
        super().__init__()
        self.features = nn.Sequential(
            nn.Linear(31, 25, bias= True),
            nn.ReLU(),
            nn.Linear(25, 20, bias= True),
            nn.ReLU(),
            nn.Dropout(p= p1),
            nn.Linear(20, 15, bias = True),
            nn.ReLU(),
            nn.Dropout(p= p2),
            nn.Linear(15, 10, bias= True),
            nn.ReLU()
        )
        self.classify = nn.Sequential(
            nn.Linear(10, 1, bias= True),
            nn.Sigmoid()
        )
    
    def forward(self, x):
        return self.classify(self.features(x))

In [3]:
class Data(Dataset):
    def __init__(self, x, y):
        super().__init__()
        self.x = torch.tensor(x.values, dtype= torch.float32)
        self.y = torch.tensor(y.values, dtype= torch.float32)
    def __len__(self):
        return self.x.shape[0]
    def __getitem__(self, index):
        x, y = self.x[index], self.y[index]
        return x, y

In [4]:
xtrain, ytrain, xtest, ytest = pd.read_csv('./X_train.csv', header= None, index_col = None), pd.read_csv('./y_train.csv', header = None, index_col = None), pd.read_csv('./X_test.csv', header = None, index_col = None), pd.read_csv('./y_test.csv', header = None, index_col = None)
train_data = Data(x= xtrain, y= ytrain)
test_data = Data(x= xtest, y= ytest)
train_loader = DataLoader(train_data, batch_size=10000, shuffle= True)
test_loader = DataLoader(test_data, batch_size= 10000)

In [5]:
model = Network(0.3, 0.6)
model.to('cuda')

Network(
  (features): Sequential(
    (0): Linear(in_features=31, out_features=25, bias=True)
    (1): ReLU()
    (2): Linear(in_features=25, out_features=20, bias=True)
    (3): ReLU()
    (4): Dropout(p=0.3, inplace=False)
    (5): Linear(in_features=20, out_features=15, bias=True)
    (6): ReLU()
    (7): Dropout(p=0.6, inplace=False)
    (8): Linear(in_features=15, out_features=10, bias=True)
    (9): ReLU()
  )
  (classify): Sequential(
    (0): Linear(in_features=10, out_features=1, bias=True)
    (1): Sigmoid()
  )
)

In [6]:
criterion = nn.BCELoss()
optimizer = torch.optim.Adam(model.parameters(), lr = 0.01, weight_decay= 0.001)

In [7]:
for epoch in range(100):
    model.train()
    running_loss = 0.0
    for input, target in train_loader:
        input, target = input.to('cuda'), target.to('cuda')
        optimizer.zero_grad()
        output = model(input)
        loss = criterion(output, target)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
    print(f'loss - {running_loss}')

loss - 64.86952823400497
loss - 33.888159096241
loss - 33.883466362953186
loss - 33.88642460107803
loss - 33.88703274726868
loss - 33.87837588787079
loss - 33.88309109210968
loss - 33.87517064809799
loss - 33.87720608711243
loss - 33.87654256820679
loss - 33.884921193122864
loss - 33.86595839262009
loss - 33.8718279004097
loss - 33.872386395931244
loss - 33.87218767404556
loss - 33.86878454685211
loss - 33.8626144528389
loss - 33.864177942276
loss - 33.85688644647598
loss - 33.86486452817917
loss - 33.86363184452057
loss - 33.8524312376976
loss - 33.81784123182297
loss - 34.048435151576996
loss - 33.889833092689514
loss - 33.88548618555069
loss - 33.88966202735901
loss - 33.88264763355255
loss - 33.899217784404755
loss - 33.89234137535095
loss - 33.89548945426941
loss - 33.89429974555969
loss - 33.889062345027924
loss - 33.869050085544586
loss - 33.84017479419708
loss - 33.83054065704346
loss - 33.81440979242325
loss - 33.816646695137024
loss - 33.824712455272675
loss - 33.822437226772

In [8]:
model.eval()
all_preds, all_labels = [], []
with torch.no_grad():
    for input, target in test_loader:
        input, target = input.to('cuda'), target.to('cuda')
        output = torch.round(model(input))
        all_preds.extend(output.to('cpu'))
        all_labels.extend(target.to('cpu'))

from sklearn.metrics import accuracy_score
print(accuracy_score(all_preds, all_labels))


0.6195944251370428
